In [1]:
!rm -rf gambiae.g?f.gz ag.db 2>/dev/null
#!wget http://www.vectorbase.org/download/anopheles-gambiae-pestbasefeaturesagamp42gtfgz -O gambiae.gtf.gz
!wget http://www.vectorbase.org/download/anopheles-gambiae-pestbasefeaturesagamp42gff3gz -O gambiae.gff.gz

--2015-02-18 14:00:45--  http://www.vectorbase.org/download/anopheles-gambiae-pestbasefeaturesagamp42gff3gz
Resolving www.vectorbase.org (www.vectorbase.org)... 129.74.255.228
Connecting to www.vectorbase.org (www.vectorbase.org)|129.74.255.228|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.vectorbase.org/download/anopheles-gambiae-pestbasefeaturesagamp42gff3gz [following]
--2015-02-18 14:00:45--  https://www.vectorbase.org/download/anopheles-gambiae-pestbasefeaturesagamp42gff3gz
Connecting to www.vectorbase.org (www.vectorbase.org)|129.74.255.228|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.vectorbase.org/sites/default/files/ftp/downloads/Anopheles-gambiae-PEST_BASEFEATURES_AgamP4.2.gff3.gz [following]
--2015-02-18 14:00:47--  https://www.vectorbase.org/sites/default/files/ftp/downloads/Anopheles-gambiae-PEST_BASEFEATURES_AgamP4.2.gff3.gz
Reusing existing connection to www.vectorbase.org:443.
HTTP

In [2]:
import gffutils
import sqlite3

In [3]:
!rm -f ag.db

In [4]:
try:
    db = gffutils.create_db('gambiae.gff.gz', 'ag.db')
except sqlite3.OperationalError:
    db = gffutils.FeatureDB('ag.db')

In [8]:
print(list(db.featuretypes()))
for feat_type in db.featuretypes():
    print(feat_type, db.count_features_of_type(feat_type))

NameError: name 'db' is not defined

In [ ]:
for contig in db.features_of_type('contig'):
    print(contig)

In [ ]:
from collections import defaultdict
num_mRNAs = defaultdict(int)
num_exons = defaultdict(int)
max_exons = 0
max_span = 0
for contig in db.features_of_type('contig'):
    cnt = 0
    for gene in db.region((contig.seqid, contig.start, contig.end), featuretype='gene'):
        cnt += 1
        span = abs(gene.start - gene.end) # strand
        if span > max_span:
            max_span = span
            max_span_gene = gene
        my_mRNAs = list(db.children(gene, featuretype='mRNA'))
        num_mRNAs[len(my_mRNAs)] += 1
        if len(my_mRNAs) == 0:
            exon_check = [gene]
        else:
            exon_check = my_mRNAs
        for check in exon_check:
            my_exons = list(db.children(check, featuretype='exon'))  # level = SLOW
            num_exons[len(my_exons)] += 1
            if len(my_exons) > max_exons:
                max_exons = len(my_exons)
                max_exons_gene = gene
    print('contig %s, number of genes %d' % (contig.seqid, cnt))
print('Max number of exons: %s (%d)' % (max_exons_gene.id, max_exons))
print('Max span: %s (%d)' % (max_span_gene.id, max_span))
print(num_mRNAs)
print(num_exons)